In [1]:
import numpy as np
import pandas as pd
import torch
import copy
import logging
import os
import time
import wandb
from pathlib import Path
import glob
import numpy as np

import click
import torch
import torch.nn as nn
import torch.optim as optim
from dotenv import find_dotenv, load_dotenv
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
import torchvision
import time
import os
import copy

plt.ion()   # interactive mode
from torchvision import datasets, models, transforms


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
from src.models.predict_model import get_most_recent_path, prepare_loaders, predict_model, get_model, test_model
from src.models.train_model_light import ResNetModule

In [4]:
data_dir = "data/processed/labeled-images/test"
model_path = get_most_recent_path("../../models")

In [5]:
print(model_path)

../../models/version=2.pth


In [6]:
model = get_model(model_path)

In [7]:
model = model.to(device)

In [8]:
data_dir_path = '../../'+ data_dir
image_dataset = datasets.ImageFolder(data_dir_path)
dataloader, dataset_size = prepare_loaders(data_dir_path)

['bbps-0-1', 'bbps-2-3', 'cecum', 'dyed-lifted-polyps', 'dyed-resection-margins', 'hemorrhoids', 'ileum', 'impacted-stool', 'polyps', 'retroflex-rectum', 'ulcerative-colitis-grade-0-1', 'ulcerative-colitis-grade-1', 'ulcerative-colitis-grade-1-2', 'ulcerative-colitis-grade-2', 'ulcerative-colitis-grade-2-3', 'ulcerative-colitis-grade-3']


/home/studio-lab-user/.conda/envs/mlops-new/lib/python3.10/site-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [9]:
class_names = image_dataset.classes

In [10]:
predictions, labels = test_model(model, dataloader, dataset_size)

In [11]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [12]:
mask_mistakes = predictions != labels

In [13]:
labels[0:100][mask_mistakes[0:100]]

tensor([0, 0], device='cuda:0')

In [14]:
# Get a batch of training data
# inputs, classes = next(iter(dataloader))

In [15]:
# inputs, classes = inputs[:4], classes[:4]

In [16]:
# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[class_names[x] for x in classes])

In [17]:
# len(inputs[predictions[0:32]!=classes])

In [18]:
i = 0
images = []
batch_size = dataloader.batch_size
wandb.init(project='hyperkvasir')
for batch in dataloader:
    print(f'{i} batch...')
    i += 1
    
    inputs, classes = batch
    inputs = inputs.to(device)
    classes = classes.to(device)
    batch_predictions = predictions[i*batch_size:(i+1)*batch_size]
    mistake_indexes = mask_mistakes[i*batch_size:(i+1)*batch_size]
    if len(classes) != len(mistake_indexes):
        print('Skip at batch {i}')
        continue
    
    mistake_classes = classes[mistake_indexes]
    mistake_predictions = batch_predictions[mistake_indexes]
    mistaken_inputs = inputs[mistake_indexes]
    
    for j in range(len(mistake_classes)):
        im = mistaken_inputs[j]
        gt = mistake_classes[j]
        pred = mistake_predictions[j]
        wandb_im = wandb.Image(im, caption=f'GT: {class_names[gt]} \nPred: {class_names[pred]}')
        images.append(wandb_im)
print(f'Logging {len(images)} mistakes...')
wandb.log({f"Mistakes": images})
wandb.finish()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mishacamry. Use `wandb login --relogin` to force relogin


/home/studio-lab-user/.conda/envs/mlops-new/lib/python3.10/site-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


0 batch...
1 batch...
2 batch...
3 batch...
4 batch...
5 batch...
6 batch...
7 batch...
8 batch...
9 batch...
10 batch...
11 batch...
12 batch...
13 batch...
14 batch...
15 batch...
16 batch...
17 batch...
18 batch...
19 batch...
20 batch...
21 batch...
22 batch...
23 batch...
24 batch...
25 batch...
26 batch...
27 batch...
28 batch...
29 batch...
30 batch...
31 batch...
32 batch...
33 batch...
34 batch...
35 batch...
36 batch...
37 batch...
38 batch...
39 batch...
40 batch...
41 batch...
42 batch...
43 batch...


44 batch...
Skip at batch {i}
45 batch...
Skip at batch {i}
Logging 111 mistakes...
